# Linear Regression Inference Using FHE
expected memory usage: 950MB.  
expected runtime: 10 seconds.

## Introduction
Linear regression is a supervised machine learning algorithm that is used to predict the value of a variable based on the value of another variable. The variable you want to predict is called the dependent variable. The variable you are using to predict the other variable's value is called the independent variable. Now we can do it fully encrypted.

## Use case
One potential FHE use case using Linear Regression is secure prediction of delivery status and can be applied to supply chain cases in a multitude of industries. 

In a supply chain, the impact of volatility can ripple throughout the entire supply chain and create delays, backlog, bottlenecks and other issues. With FHE, third-party logistics (3PL) providers can securely pool their own historical shipment information (e.g. source, destination, distance between them, mode of shipment transportation, etc.) in a Multi Enterprise Business Network (MEBN) to conduct secure predictions of delivery status and determine, in seconds, whether a shipment will be on time or not.  The reason we do this with FHE is because we expect that the 3PL providers would have access to sensitive data from across many different clients. 

<br>

#### Step 1. Import pyhelayers

In [ ]:
import numpy as np
import sys
import pyhelayers
import json
import os
import time
import utils

utils.verify_memory()

print("misc. init ready")

<br>

#### Step 2. Generate the plain model and save it into a file

In [ ]:
# Create model data
dims=39
batch_size=4
coefs=np.random.randn(1,dims)
intercept=[0.1]

# Save to json file
model_json={}
model_json['coef_']=coefs.tolist()
model_json['intercept_']=intercept

data_dir = os.path.join('data', 'linear_reg')
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

model_file=os.path.join(data_dir,'model.json')

with open(model_file, 'w') as mf:
    json.dump(model_json, mf)
    

# Populate plain model object
hyper_params = pyhelayers.PlainModelHyperParams()
hyper_params.logistic_regression_activation = pyhelayers.LRActivation.NONE
plain = pyhelayers.PlainModel.create(hyper_params, [model_file])
print(plain)

<br>

#### Step 3. Compile the plain model

In [ ]:
he_run_req = pyhelayers.HeRunRequirements()
he_run_req.set_he_context_options([pyhelayers.DefaultContext()])

he_run_req.optimize_for_batch_size(batch_size)

profile = pyhelayers.HeModel.compile(plain, he_run_req)

context = pyhelayers.HeModel.create_context(profile)

print("He profile ready and context initialized", context)


<br>

#### Step 4. Construct an encrypted LinearRegression model as a LogisticRegression model with the activation function set to NONE

In [ ]:
# Encrypt model
enc_model=pyhelayers.LogisticRegression(context)
enc_model.encode_encrypt(plain, profile)
print("Model encrypted")

<br>

#### Step 5. Run prediction and report results

In [ ]:
iop = enc_model.create_io_processor()

duration=0
for i in range(3):
    input=np.random.randn(batch_size,dims)

    enc_input = pyhelayers.EncryptedData(context)
    iop.encode_encrypt_inputs_for_predict(enc_input, [input])

    start_time = time.perf_counter()
    enc_res = pyhelayers.EncryptedData(context)
    enc_model.predict(enc_res, enc_input)
    duration += time.perf_counter() - start_time

    res=iop.decrypt_decode_output(enc_res)

    res=res.reshape(batch_size)
    expected_res=np.inner(coefs,input)+intercept
    expected_res = expected_res[0]
    print("         Result",res)
    print("Expected result",expected_res)
    utils.report_duration("predict time (avg.)",duration/(i+1))
    print("")

In [ ]:
print("RAM usage:", utils.get_used_ram(), "MB")